In [238]:
def writeaxioms(temp_graph_dictionary):




    nodes = ""
    #for node in range(0, len(temp_graph_dictionary)):
    for node in temp_graph_dictionary:
        nodes = nodes + "a" + str(node["Node"]) + ";"


    nodes = nodes[:-1]
    #nodes = "aS;a0;a1;a2;a3;a4;a5;a6;a7;a8;a9;a10;a11;a12;a13;a14;a15;a16;a17;a18;a19;a20;a21;a22;a23;a24;a25;a26;a27;a28;a29;a30;a31;a32;a33;a34;a35;a36;a37;a38;a39;a40;a41;a42;a43;a44;a45;a46;a47;a48;a49;a50;aF"

    f = open("logicalprogram.lp", "w")
    f.write("""

    event(""" + nodes + """).
    fluent(started(E)) :- event(E).
    fluent(completed(E)) :- event(E).
    time(1..30).

    % definitions
    initiates(E, started(E), T) :- happens(E,T), event(E), time(T).
    terminates(E, started(E1), T) :- happens(E,T), holdsAt(started(E1),T), event(E), time(T).
    initiates(E, completed(E1), T) :- happens(E,T), holdsAt(started(E1), T), event(E), time(T).

    holdsAt(F,T) :- happens(E,T1), initiates(E,F,T1), not stoppedIn(T1,F,T), T1<T, time(T), time(T1).
    stoppedIn(T1,F,T2) :- happens(E,T), T1<T, T<T2, terminates(E,F,T), time(T), time(T1), time(T2).

    % complements
    predecesor(E1, E2, T) :- happens(E1,T1), happens(E2, T), T1 = (T-1), time(T), time(T1), event(E1), event(E2).
    another(E, T1, T2) :- happens(E, T), T1 < T, T < T2, time(T), time(T1), time(T2), event(E).
    bbetween(E1, E2, T) :- happens(E1, T1), happens(E2, T2), happens(E2, T), not another(E1, T1, T), T1 < T2, T2 < T, time(T), time(T1), time(T2), event(E1), event(E2).
    
    % restrictions

    :- happens(E, T), happens(E1, T), event(E), event(E1), E != E1.


    """)

    f.close()

#print(writeaxioms(graph_1))

Add input constraints for "or" function for the graph given to the function

In [239]:
def function_sequence(Node, Inputs):
    str1 = ":- happens(a"+str(Node)+",T), not predecesor(a"+str(Inputs[0])+",a"+str(Node)+",T), time(T)."
    return str(str1)

In [240]:
def function_or(Node, Inputs): #node is the node number, inputs is the array of inputs
    strN = ":- happens(a" + str(Node) + ",T), not 1 {" 
    for input in Inputs:
        strN = strN + "predecesor(a"+ str(input) + ",a" + str(Node) + ",T); "   
    
    strN = strN[:-2]
    strN = strN + "}."
    return str(strN)

In [241]:
def addinputsOR(temp_graph_dictionary):
    for node in temp_graph_dictionary:
            if len(node["Inputs"]) == 0: #in case there are no inputs, it must be the starting node or a node without inputs
                #print("no inputs to " + str(node["Node"]) + " from " + str(node["Inputs"])  )
                #print(node["Inputs"])
                unusedvariable = 0
            if len(node["Inputs"]) == 1:
                f = open("logicalprogram.lp", "a")
                f.write(function_sequence(node["Node"], node["Inputs"]) + "\n")
                f.close()
            if len(node["Inputs"]) >= 2:
                f = open("logicalprogram.lp", "a")
                f.write(function_or(node["Node"], node["Inputs"]) + "\n")
                f.close()

Now add the sequence based on the input sequence:

In [242]:
def addsequencefromlist(i):
    sequence = "\n"+ "\n"+ "\n" + "% sequence" + "\n"+ "\n"
    node = 0
    #print("current sequence: ", i)
    for action in i:
        node = node + 1
        sequence = sequence + "happens(a"+str(action)[1:]+","+str(node)+")."  + "\n"
    f = open("logicalprogram.lp", "a")
    f.write(sequence)
    f.close()

Check if graph the logical program is satisfiable



In [243]:
import subprocess

def satisfiable():
    output = subprocess.getoutput("clingo logicalprogram.lp 0")
    #print(output)
    returnvalue = not "UNSATISFIABLE" in output
    #print(returnvalue)
    #print(output)
    return returnvalue

This script should receive a graph (dictionary) and a sequence (list) and it will return true or false whether the graph can implement the sequence

In [244]:
def sequence_check(sequence, graph):
    writeaxioms(graph) #write the initial axioms
    addinputsOR(graph) #add the required input constraints for nodes
    addsequencefromlist(sequence) #add the sequence that has to be verified
    return satisfiable() #check that the file accepts the graph currently


In [245]:
#testing

unused_graph_test = [    
    {"Node": 1, "Edges": [5], "Processed": False, "Inputs": []},
    {"Node": 5, "Edges": [8], "Processed": False, "Inputs": [1]},
    {"Node": 8, "Edges": [2], "Processed": False, "Inputs": [5]},
    {"Node": 2, "Edges": [3], "Processed": False, "Inputs": [8,4]},
    {"Node": 3, "Edges": [6], "Processed": False, "Inputs": [2]},
    {"Node": 6, "Edges": [9,4,11], "Processed": False, "Inputs": [3]},
    {"Node": 4, "Edges": [2], "Processed": False, "Inputs": [9,6]},
    {"Node": 11, "Edges": [], "Processed": False, "Inputs": [6]},
    {"Node": 9, "Edges": [4], "Processed": False, "Inputs": [6]},
    ]

#sequence_test = ["G11","G5","G8","G2","G3","G6","G1"]

#print(sequence_check(sequence_test, unusued_graph_test))

Get the list of sequences into the sequencelist variable



In [246]:
sequencelist = []


sequence_test = ["GS", "G4", "G1","G5","G5","G8","G2","G3","G6","G1","GF"]

sequencelist = sequence_test

Get the graph into a graph list of dictionares of form [ {Node: int, Edges: [int], Processed: bool, Inputs [int]} ]

In [247]:
graph = []

graph_2 = [
    {"Node": "S", "Edges": ["1"], "Processed": False, "Inputs": []},
    {"Node": "1", "Edges": ["5"], "Processed": False, "Inputs": ["S"]},
    {"Node": "5", "Edges": ["8"], "Processed": False, "Inputs": ["1"]},
    {"Node": "8", "Edges": ["2"], "Processed": False, "Inputs": ["5"]},
    {"Node": "2", "Edges": ["3"], "Processed": False, "Inputs": ["8","4"]},
    {"Node": "3", "Edges": ["6"], "Processed": False, "Inputs": ["2"]},
    {"Node": "6", "Edges": ["9","4","11"], "Processed": False, "Inputs": ["3"]},
    {"Node": "4", "Edges": ["2"], "Processed": False, "Inputs": ["9","6"]},
    {"Node": "11", "Edges": ["F"], "Processed": False, "Inputs": ["6"]},
    {"Node": "9", "Edges": ["4"], "Processed": False, "Inputs": ["6"]},
    {"Node": "F", "Edges": [], "Processed": False, "Inputs": ["11"]},
    ]




graph_1 = [
    {"Node": "S", "Edges": ["1"], "Processed": False, "Inputs": []},
    {"Node": "1", "Edges": ["5"], "Processed": False, "Inputs": ["S"]},
    {"Node": "5", "Edges": ["8"], "Processed": False, "Inputs": ["1"]},
    {"Node": "8", "Edges": ["2"], "Processed": False, "Inputs": ["5"]},
    {"Node": "2", "Edges": ["3"], "Processed": False, "Inputs": ["8","4"]},
    {"Node": "3", "Edges": ["6"], "Processed": False, "Inputs": ["2"]},
    {"Node": "6", "Edges": ["4","11"], "Processed": False, "Inputs": ["3"]},
    {"Node": "4", "Edges": ["2"], "Processed": False, "Inputs": ["6"]},
    {"Node": "11", "Edges": ["F"], "Processed": False, "Inputs": ["6"]},
    {"Node": "F", "Edges": [], "Processed": False, "Inputs": ["11"]},

    ]



graph_3 = [
    {"Node": "S", "Edges": ["1"], "Processed": False, "Inputs": []},
    {"Node": "1", "Edges": ["5"], "Processed": False, "Inputs": ["S"]},
    {"Node": "5", "Edges": ["8"], "Processed": False, "Inputs": ["1"]},
    {"Node": "8", "Edges": ["2"], "Processed": False, "Inputs": ["5"]},
    {"Node": "2", "Edges": ["3"], "Processed": False, "Inputs": ["8","4"]},
    {"Node": "3", "Edges": ["6"], "Processed": False, "Inputs": ["2"]},
    {"Node": "6", "Edges": ["9","11"], "Processed": False, "Inputs": ["3"]},
    {"Node": "4", "Edges": ["2"], "Processed": False, "Inputs": ["9"]},
    {"Node": "11", "Edges": ["F"], "Processed": False, "Inputs": ["6"]},
    {"Node": "9", "Edges": ["4"], "Processed": False, "Inputs": ["6"]},
    {"Node": "F", "Edges": [], "Processed": False, "Inputs": ["11"]},
    ]



graph = unused_graph_test

Now create a loop that goes through a list and a given graph and for each list entry checks satisfiability, if true - the element goes into the list of accepted elements, if false - the element goes into a list of not accepted elements. 

In [248]:
def get_two_lists(sequencelist,graph):
    acceptedlist = []
    notacceptedlist = []
    allowednodes = []
    for dictitem in graph:
        allowednodes.append('G' + dictitem["Node"])
    #print(allowednodes)

    for i in sequencelist:
        if i in allowednodes:
            acceptedlist.append(i)
            #print(acceptedlist)
            if sequence_check(acceptedlist, graph):
                unusedvariable = 0
            else: 
                acceptedlist.pop()
                notacceptedlist.append(i)
        else:
            notacceptedlist.append(i)
        
    return [acceptedlist,notacceptedlist]



In [249]:
#testing function
# #print(get_two_lists(sequencelist,graph_1))
#print(get_two_lists(sequencelist,graph_2))

Import the sequences

In [250]:
import csv
#make a list of disctionaries of sequences
seqlist = []
with open('suturing_file_sequences.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        seqlist.append({"sequence": row})


#make a list of dictionaries of filename
filelist = []
with open('suturing_files.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        stringarray = ""
        for i in row:
            stringarray = stringarray + i
        #print(stringarray)
        filelist.append({"filename": stringarray})

#print(filelist)

#merge two lists of dictionaries
for i in range (len(filelist)):
    seqlist[i]["filename"] = filelist[i]["filename"]

function for adding starting and ending nodes

In [251]:
def add_start_end(seqlist):
    seqlist.insert(0,"GS")
    seqlist.append("GF")

In [252]:
#add_start_end(seqlist[10]["sequence"])
#print(seqlist[10]["sequence"])
#print(get_two_lists(seqlist[10]["sequence"],graph_1))

In [253]:
graph1counter = 0
graph2counter = 0
graph3counter = 0
avg1 = []
avg2 = []
avg3 = []
oavg1 = []
oavg2 = []
oavg3 = []
check1and2 = 0
check1and3 = 0
check2and3 = 0
heck1and2and3 = 0
graph1_acc_list = []
graph2_acc_list = []
graph3_acc_list = []

#print(get_two_lists(seqlist[11]["sequence"],graph_1))

for listitem in seqlist:

#    graph1counter = graph1counter + 1
#    #print(add_start_end(listitem["sequence"]))
    add_start_end(listitem["sequence"])
#    #print(listitem["sequence"])
#    print("graph 1 lists")
#    #print(get_two_lists(listitem["sequence"],graph_1)[0])
    seq1 = get_two_lists(listitem["sequence"],graph_1)
#    print(listitem)
#    print(seq1)
   
   
    if seq1[0][-1] == "GF" and len(seq1[1]) == 0:
       graph1counter = graph1counter + 1
       #avg1.append(len(seq1[1])/(len(seq1[0]) + len(seq1[1])))


#    #avg1.append(len(seq1[0])/(len(seq1[0]) + len(seq1[1])))
   
   
   
    if seq1[0][-1] == "GF":
       graph1_acc_list.append(1)
    else:
       graph1_acc_list.append(0)


   #    #graph1counter = graph1counter + 1
   #    #avg1.append(len(get_two_lists(listitem["sequence"],graph_1)[0])/(len(get_two_lists(listitem["sequence"],graph_1)[0]) + len(get_two_lists(listitem["sequence"],graph_1)[1])))

   # #print("graph 2 lists")
   # #print(get_two_lists(listitem["sequence"],graph_2)[0])
    seq2 = get_two_lists(listitem["sequence"],graph_2)
    if seq2[0][-1] == "GF" and len(seq2[1]) == 0:
       #avg2.append(len(seq2[1])/(len(seq2[0]) + len(seq2[1])))
      graph2counter = graph2counter + 1



    if seq2[0][-1] == "GF":
       graph2_acc_list.append(1)
    else:
       graph2_acc_list.append(0)

   # # if seq2[0][-1] != "GF":
   # #    graph1counter = graph1counter + 1
   # #    avg2.append(len(get_two_lists(listitem["sequence"],graph_2)[1])/(len(get_two_lists(listitem["sequence"],graph_2)[0]) + len(get_two_lists(listitem["sequence"],graph_2)[1])))

   # #print("graph 3 lists")
   # #print(get_two_lists(listitem["sequence"],graph_3)[0])
    seq3 = get_two_lists(listitem["sequence"],graph_3)
    if seq3[0][-1] == "GF"  and len(seq3[1]) == 0:
      graph3counter = graph3counter + 1
      #avg3.append(len(seq3[1])/(len(seq3[0]) + len(seq3[1])))
       
   
   
    if seq3[0][-1] == "GF":
       graph3_acc_list.append(1)
    else:
       graph3_acc_list.append(0)

   # if seq3[0][-1] != "GF":
   #    graph1counter = graph1counter + 1
   #    avg3.append(len(get_two_lists(listitem["sequence"],graph_3)[1])/(len(get_two_lists(listitem["sequence"],graph_3)[0]) + len(get_two_lists(listitem["sequence"],graph_3)[1])))


#    if seq1[0][-1] != "GF":
#       print("sequence accepted by 1")
#       unusedvariable = 0
#       if seq2[0][-1] != "GF":
      
#          check1and2 = check1and2 + 1
#          print("Sequences accepted by graphs 1 and 2 ", check1and2)
#          if seq3[0][-1] != "GF":
#                check1and2and3 = check1and2and3 + 1
#                print("Sequences accepted by graphs 1 and 2 and 3 ", check1and2and3)
#       if seq3[0][-1] != "GF":
#          check1and3 = check1and3 + 1
#          print("Sequences accepted by graphs 1 and 3 ", check1and3)
   
#    if seq3[0][-1] != "GF":
#       print("sequence accepted by 3")
#       if seq2[0][-1] != "GF":
#          check2and3 = check2and3 + 1
#          print("Sequences accepted by graphs 2 and 3 ", check2and3)

# print("Sequences rejected by graphs 1 and 2 ", check1and2)
# print("Sequences rejected by graphs 1 and 3 ", check1and3)
# print("Sequences rejected by graphs 2 and 3 ", check2and3)
# print("Sequences rejected by graphs 1 and 2 and 3 ", check1and2and3)
#print(avg1, ' ', avg2, ' ', avg3, ' ')


print(graph1_acc_list)
print(graph2_acc_list)
print(graph3_acc_list)

print(graph1counter)
print(graph2counter)
print(graph3counter)



[1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1]
[1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1]
[1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1]
7
9
1


In [254]:
yes1and2acc = 0
yes1and3acc = 0
yes2and3acc = 0
yes2and3acc = 0
yes1and2and3acc = 0
no1and2acc = 0
no1and3acc = 0
no2and3acc = 0
no2and3acc = 0
no1and2and3acc = 0

for i in range(0,len(graph1_acc_list)):
    if graph1_acc_list[i] == graph2_acc_list[i] == 1:
        yes1and2acc = yes1and2acc + 1
    if graph1_acc_list[i] == graph3_acc_list[i] == 1:
        yes1and3acc = yes1and3acc + 1
    if graph2_acc_list[i] == graph3_acc_list[i] == 1:
        yes2and3acc = yes2and3acc + 1
    if graph1_acc_list[i] == graph2_acc_list[i] == graph3_acc_list[i] == 1:
        yes1and2and3acc = yes1and2and3acc + 1 
    
    if graph1_acc_list[i] == graph2_acc_list[i] == 0:
        no1and2acc = no1and2acc + 1
    if graph1_acc_list[i] == graph3_acc_list[i] == 0:
        no1and3acc = no1and3acc + 1
    if graph2_acc_list[i] == graph3_acc_list[i] == 0:
        no2and3acc = no2and3acc + 1
    if graph1_acc_list[i] == graph2_acc_list[i] == graph3_acc_list[i] == 0:
        no1and2and3acc = no1and2and3acc + 1 


print('1 and 2 ', yes1and2acc, '1 and 3 ', yes1and3acc, "2 and 3 ", yes2and3acc, "1 and 2 and 3 ", yes1and2and3acc)
print('1 and 2 ', no1and2acc, '1 and 3 ', no1and3acc, "2 and 3 ", no2and3acc, "1 and 2 and 3 ", no1and2and3acc)

print("mean 1 ", sum(graph1_acc_list)/39)
print("mean 2", sum(graph2_acc_list)/39)
print("mean3 ", sum(graph3_acc_list)/39)
print("mean all: ", (sum(graph1_acc_list) + sum(graph2_acc_list) + sum(graph3_acc_list))/(3*39)) 

1 and 2  18 1 and 3  18 2 and 3  18 1 and 2 and 3  18
1 and 2  19 1 and 3  18 2 and 3  20 1 and 2 and 3  18
mean 1  0.5128205128205128
mean 2 0.46153846153846156
mean3  0.48717948717948717
mean all:  0.48717948717948717


In [255]:
from statistics import mean
print(mean(avg1), mean(avg2), mean(avg3), (mean(avg1) + mean(avg2) + mean(avg3))/3)

StatisticsError: mean requires at least one data point

In [ ]:
print(graph1_acc_list)
print(graph2_acc_list)
print(graph3_acc_list)
print(avg1,avg2,avg3)

[1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1]
[1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1]
[1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1]
[0.3225806451612903, 0.0, 0.0, 0.045454545454545456, 0.0, 0.0, 0.045454545454545456, 0.125, 0.19230769230769232, 0.16, 0.35, 0.0, 0.25, 0.045454545454545456, 0.125, 0.045454545454545456, 0.0, 0.15, 0.08695652173913043, 0.0] [0.3225806451612903, 0.0, 0.0, 0.045454545454545456, 0.0, 0.0, 0.045454545454545456, 0.0, 0.35, 0.0, 0.25, 0.0, 0.08333333333333333, 0.045454545454545456, 0.0, 0.15, 0.08695652173913043, 0.0] [0.7096774193548387, 0.5714285714285714, 0.5714285714285714, 0.5909090909090909, 0.5714285714285714, 0.5714285714285714, 0.5909090909090909, 0.0, 0.55, 0.55, 0.5714285714285714, 0.6785714285714286, 0.36363636363636365, 0.208333333